In [1]:
# 重采样及频率转换
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
from pandas import DataFrame, Series
from pandas.tseries.offsets import Day, Hour, Minute, MonthEnd

In [2]:
rng = pd.date_range('1/1/2000', periods=100, freq='D')
ts = Series(np.random.randn(len(rng)), index=rng)
ts.resample('M', kind='period').mean() # 统计每月平均数，实际上是根据月份做了group。

2000-01   -0.006518
2000-02    0.017119
2000-03   -0.113910
2000-04    0.327016
Freq: M, dtype: float64

In [3]:
# resample方法的参数
# freq：
# how已经被淘汰。原来的how='mean'写成resample(...).mean()。
# axis：
# fill_method已经被淘汰。原来的fill_method='...'写成resample(...).ffill()。方法为ffill或bfill，默认不插值。
# close：      在降采样中，各时间段的哪一端是闭合的，'right'（默认）或'left'。
# label：      在降采样中，如何设置聚合值的标签。'right'（默认）或'left'（面元的右/左边界）。
#              例如，9:30到9:35之间的这5分钟会被标记成9:30或9:35。
# loffset：    面元标签的时间校正值。比如'-1s'或Second(-1)用于将聚合标签调早1秒。
# limit：      在向前或向后填充时，允许填充的最大时期数。默认None。
# kind：       聚合到时期（'period'）或时间戳（'timestamp'）,默认聚合到时间序列的索引类型。
# convertion： 当重采样时期时，将低频转换到高频采样所采用的约定（'start'或'end'），默认'end'。

In [4]:
# 降采样

In [5]:
rng = pd.date_range('1/1/2000', periods=12, freq='T') # T是按分钟
ts = Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int32

In [6]:
ts.resample(Minute(5)).sum() # 等价：ts.resample('5min').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int32

In [7]:
ts.resample('5min', closed='right').sum() # 比较一下left/right不同结果

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

In [8]:
ts.resample('5min', closed='left', label='right').sum() # 比较close/label不同组合的结果

2000-01-01 00:05:00    10
2000-01-01 00:10:00    35
2000-01-01 00:15:00    21
Freq: 5T, dtype: int32

In [9]:
ts.resample('5min', loffset='-1s').sum()

1999-12-31 23:59:59    10
2000-01-01 00:04:59    35
2000-01-01 00:09:59    21
Freq: 5T, dtype: int32

In [10]:
# OHLC重采样 Open/High/Low/Close

In [11]:
ts.resample('5min').ohlc() # 其实就是5分钟k线图

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


In [12]:
# 通过groupby进行重采样

In [13]:
rng = pd.date_range('1/1/2000', periods=100, freq='D')
ts = Series(np.arange(100), index=rng)
ts.groupby(lambda x: x.month).mean()

1    15
2    45
3    75
4    95
dtype: int32

In [14]:
ts.groupby(lambda x: x.weekday).mean() # 0是周日

0    47.5
1    48.5
2    49.5
3    50.5
4    51.5
5    49.0
6    50.0
dtype: float64

In [15]:
# 升采样和插值

In [16]:
frame = DataFrame(np.random.randn(2, 4),
                  index=pd.date_range('1/1/2000', periods=2, freq='W-WED'),
                  columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-2.366333,0.691709,-0.158442,2.562437
2000-01-12,-0.207933,-0.394861,0.264470,-1.361815


In [17]:
df_daily = frame.resample('D') # 注意警告信息，resample现在生成的是一个延迟计算对象。

In [18]:
for i in df_daily.iteritems():
    print(i)

('Colorado', 2000-01-05   -2.366333
2000-01-06         NaN
2000-01-07         NaN
2000-01-08         NaN
2000-01-09         NaN
2000-01-10         NaN
2000-01-11         NaN
2000-01-12   -0.207933
Freq: D, Name: Colorado, dtype: float64)
('Texas', 2000-01-05    0.691709
2000-01-06         NaN
2000-01-07         NaN
2000-01-08         NaN
2000-01-09         NaN
2000-01-10         NaN
2000-01-11         NaN
2000-01-12   -0.394861
Freq: D, Name: Texas, dtype: float64)
('New York', 2000-01-05   -0.158442
2000-01-06         NaN
2000-01-07         NaN
2000-01-08         NaN
2000-01-09         NaN
2000-01-10         NaN
2000-01-11         NaN
2000-01-12    0.264470
Freq: D, Name: New York, dtype: float64)
('Ohio', 2000-01-05    2.562437
2000-01-06         NaN
2000-01-07         NaN
2000-01-08         NaN
2000-01-09         NaN
2000-01-10         NaN
2000-01-11         NaN
2000-01-12   -1.361815
Freq: D, Name: Ohio, dtype: float64)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: 
.resample() is now a deferred operation
You called iteritems(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead
  if __name__ == '__main__':


In [19]:
df_daily = frame.resample('D', ).ffill() # ffill做填充，所以resample必须先展开计算。
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-2.366333,0.691709,-0.158442,2.562437
2000-01-06,-2.366333,0.691709,-0.158442,2.562437
2000-01-07,-2.366333,0.691709,-0.158442,2.562437
2000-01-08,-2.366333,0.691709,-0.158442,2.562437
2000-01-09,-2.366333,0.691709,-0.158442,2.562437
2000-01-10,-2.366333,0.691709,-0.158442,2.562437
2000-01-11,-2.366333,0.691709,-0.158442,2.562437
2000-01-12,-0.207933,-0.394861,0.264470,-1.361815


In [20]:
df_daily = frame.resample('D', limit=2).ffill() # 最多填充2次
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-2.366333,0.691709,-0.158442,2.562437
2000-01-06,-2.366333,0.691709,-0.158442,2.562437
2000-01-07,-2.366333,0.691709,-0.158442,2.562437
2000-01-08,-2.366333,0.691709,-0.158442,2.562437
2000-01-09,-2.366333,0.691709,-0.158442,2.562437
2000-01-10,-2.366333,0.691709,-0.158442,2.562437
2000-01-11,-2.366333,0.691709,-0.158442,2.562437
2000-01-12,-0.207933,-0.394861,0.264470,-1.361815


In [21]:
# 通过时期进行重采样

In [22]:
frame = DataFrame(np.random.randn(24, 4),
                  index=pd.period_range('1-2000', '12-2001', freq='M'),
                  columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame.head()

,Colorado,Texas,New York,Ohio
2000-01,0.833519,0.378349,1.651265,0.218372
2000-02,0.588505,0.358069,-1.187257,-0.352007
2000-03,-1.429854,-2.381661,-0.151033,-0.454657
2000-04,-0.645234,-1.356728,-0.148877,-0.185001
2000-05,0.855779,1.392728,-0.656088,1.413772


In [23]:
annual_frame = frame.resample('A-DEC').mean()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.229285,-0.228343,-0.091366,0.303724
2001,-0.029608,-0.584833,0.031789,0.027464


In [24]:
annual_frame.resample('Q-DEC').ffill()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.229285,-0.228343,-0.091366,0.303724
2001,-0.029608,-0.584833,0.031789,0.027464


In [25]:
annual_frame.resample('Q-DEC', convention='start').ffill()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.229285,-0.228343,-0.091366,0.303724
2001,-0.029608,-0.584833,0.031789,0.027464


In [26]:
# 一些讲解：
# - 在降采样中，目标频率必须是源频率的子时期（subperiod）。降采样 <-> 高频到低频
# - 在升采样中，目标频率必须是源频率的超时期（superperiod）。升采样 <-> 低频到高频
# - 如果不满足这些条件，就会引发异常。这主要影响的是按季、年、周计算的频率。
#   例如，由Q-MAR定义的时间区间只能升采样为A-MAR、A-JUN、A-SEP、A-DEC等。(有问题自己慢慢琢磨吧...)

In [27]:
quater_frame = annual_frame.resample('Q-MAR').ffill()
quater_frame

,Colorado,Texas,New York,Ohio
2000Q4,-0.229285,-0.228343,-0.091366,0.303724
2001Q1,-0.229285,-0.228343,-0.091366,0.303724
2001Q2,-0.229285,-0.228343,-0.091366,0.303724
2001Q3,-0.229285,-0.228343,-0.091366,0.303724
2001Q4,-0.029608,-0.584833,0.031789,0.027464
2002Q1,-0.029608,-0.584833,0.031789,0.027464
2002Q2,-0.029608,-0.584833,0.031789,0.027464
2002Q3,-0.029608,-0.584833,0.031789,0.027464
